In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("image save:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## **Decision Tree(의사결정 나무)**

Rule 기반 의사결정 모델

* Binary Question(이진 질의)의 분류 규칙을 바탕으로 Root Node의 질의 결과에 따라 Branch(가지)를 타고 이동

* 최종적으로 분류 또는 예측값을 나타내는 Leaf까지 도달함

#### 특징

* 지도 학습 알고리즘의 일종
* Rule은 Entropy가 가장 작은 방향으로 학습
* 종류
  * 범주형 : Classification Tree, Entropy가 낮은 방향으로 Rule 결정
  * 수치형 : Regression Tree, Variance가 낮은 방향으로 Rule 결정
* 용어
  * Root Node : 최상위 노드
    * Splitting : 하위 노드로 분리되는 행위
    * Branch : 노드들의 연결(Sub-Tree)
  * Decision Node : 2개의 하위노드로 분리되는 노드
    * Parent Node : 분리가 발생하는 노드
  * Leaf(Terminal Node) : 더이상 분리되지 않는 최하위 노드
    * Child Node : 분리가 발생한 후 생성되는 노드

#### Model Capacity in Decision Tree

* 의사결정 나무는 규칙 기반으로 직관적으로 이해하기 쉽고, 설명력(Model Capacity)이 좋은 알고리즘
* 각 노드별 불순도(Impurity, = Entropy)에 기반한 최적의 분류 규칙을 적용함
  * Splitting 과정을 반복하면서 의사결정 나무가 성장하며 모델의 설명력이 증가함
  * 나무가 성장하면서 설명력이 증가하면 과적합이 발생할 수 있음
* Leaf는 순도(동질성)이 높은 적은 수의 데이터 포인트를 포함함
  * 순도 높은 결과를 만들기 위해 순도 높은 Leaf가 나올 때까지 Recursive Partitioning을 수행함
  * Leaf들의 불순도는 0

#### Overfitting(과적합)
* 노드들은 불순도가 낮은 방향으로 분리됨
* 노드들이 분리되면서 너무 복잡하고 큰 의사결정나무 모델을 생성하여 과적합 문제가 발생됨

#### Pruning(가지치기)

* 과적합 예방 및 모델 성능향상 목적으로 의사결정나무의 파라미터를 조정하는 방법(Hyper Parameter)
* 보통 사후에 진행하고, 모델을 경량화할 수 있는 방법
* max_depth : 의사결정나무의 성장 깊이 지정
* min_samples_leaf : 리프에 들어가는 최소 샘플의 개수 지정

#### 불순도(= Entropy) 계산 방식

트리 모델에서는 불순도(Impurity)가 낮은 방식으로 트리가 만들어지는데, Entropy와 Gini Index가 있다.

#### 1) Entropy

* 분리 정보 이득(질문 전 Entropy - 질문 후 Entropy(불순도))이 큰 특징으로 분리 발생
* 분리 정보 이득을 비교하여 이득이 큰 Feature로 분리 발생
$
-(\sum\limits_{i=1}^n p_i \log_2 p_i ) $

* 분리 정보 이득 계산 예
  * ‘소득’ Feature로 분리해야할지, ‘학생’ Feature로 분리해야할지 분리 정보 이득을 계산하여 이득이 큰 쪽으로 노드 분리
  * ‘학생’ Feature 계산은 생략

In [2]:
'''
학생	  소득	연체
네	    없음	Yes
아니오  없음	No
네	    없음	Yes
아니오	있음	No
네	    없음	Yes
아니오	없음	No
'''

'\n학생\t  소득\t연체\n네\t    없음\tYes\n아니오  없음\tNo\n네\t    없음\tYes\n아니오\t있음\tNo\n네\t    없음\tYes\n아니오\t없음\tNo\n'

1) (소득)질문 후 Entropy 계산
  * $p(\text{있음}) \times E(Yes, No) + p(\text{없음}) \times E(Yes, No)$

    $= \frac{1}{6} \times E(0, 1) + \frac{5}{6} \times E(3, 2)$

    $=\frac{1}{6} \times (-\frac{0}{1} \times \log_2\frac{0}{1} - \frac{1}{1} \times \log_2\frac{1}{1}) + \frac{5}{6} \times (-\frac{3}{5} \times \log_2\frac{3}{5} - \frac{2}{5} \times \log_2\frac{2}{5})$
    
    $= 0.966$

2) (소득)분리 정보 이득 계산
  * $1 − 0.966 = 0.034$

3) (학생)분리 정보 이득이 1이므로 ‘학생’ Feature로 노드가 분리됨

#### 2) Gini Impurity Index

* sklearn의 기본 불순도 계산 방식
* 지니 불순도 지수(1 - 특징 지니 지수)가 작은 특징으로 분리 발생(분리 정보 이득과 반대 개념)
* 특징 지니 지수가 클 수록 불순도가 낮음
$
1-\sum\limits_{i=1}^n {p_i}^2 $

* 지니 불순도 지수 계산 예
  * ‘소득’ Feature로 분리해야할지, ‘학생’ Feature로 분리해야할지 지니불순도 지수를 계산해 작은쪽으로 노드 분리
  * ‘학생’ Feature 계산은 생략

In [3]:
'''
학생	  소득	연체
네	    없음	Yes
아니오	없음	 No
네	    없음	Yes
아니오	있음	 No
네	    없음	Yes
아니오	없음	 No
'''

'\n학생\t  소득\t연체\n네\t    없음\tYes\n아니오\t없음\t No\n네\t    없음\tYes\n아니오\t있음\t No\n네\t    없음\tYes\n아니오\t없음\t No\n'

1) (소득)특징 지니 지수 계산

있음 : ${\frac{0}{1}}^2 + {\frac{1}{1}}^2 = 1$

없음 : ${\frac{3}{5}}^2 + {\frac{2}{5}}^2 = 0.52$

특징 지니 지수 : $\frac{1}{6} * 1 + \frac{5}{6} * 0.52 $

2) (소득)지니 불순도 지수 계산

$1 − 0.6 = 0.4$

#### Feature Importance

모델에 대한 Feature의 기여도(중요도)

트리 모델 혹은 트리 기반의 앙상블 모델에서는 Feature Importance를 확인할 수 있음(Random Forest, …)

* 1 : 기여도 높음 / 0 : 기여도 없음
* Feature Importance가 0이라 할지라도, 직접 모델에 적용시켜봐야 실제 기여도를 알 수 있음

## **GridSearchCV(for Hyparameter Tuning)**

sklearn에서 Hyperparameter Tuning 시 사용 되는 방법

입력한 parameters의 경우의 수를 모두 모델에 적용하여, 지정한 score가 높은 parameters의 조합을 찾아내는 클래스

* 무조건 모델이 최적화 되었다고 맹신할 수는 없음
* 모든 파라미터들의 조합을 확인하므로 시간이 오래 걸림

## **교차 검증(Cross Validation)**

#### Overfitting 방지를 위해 수행되는 검증 방법

* 다양하게 Training Data와 Validation Data를 변경하면서 Model Validation 수행
* Validation을 한 번만 수행하면 특정 Data에만 최적화 될수 있기 때문에 교차 검증 실시

#### K-Fold Cross Validation

Training Data를 무작위로 균등하게 K개 그룹으로 나누어서 검증하는 방법

최적의 파라미터를 찾기 위한 방법 중 하나

In [ ]:
filename = "K-fold.png"
images_path = os.path.join(PROJECT_ROOT_DIR, "images")
show_img = mpimg.imread(os.path.join(images_path, filename))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.imshow(show_img)

#### 과정
* Training Data를 K개 그룹으로 나눔
* 각 그룹(Training Data)을 (K-1) 개의 Training Fold와 1개의 Validation Fold로 나눔
* Training Fold로 학습을 진행하고, Validation Fold에 대해 성능을 측정함
* 총 K개 그룹 결과의 평균을 측정하여 모델의 최적 parameter를 찾음
* 최적 parameter로 모델을 학습시킨 후 Test Data에 검증을 수행함

#### 특징
  * K : Hyperparameter
    * 일반적으로 5 ~ 10 사용
  * Data가 충분히 많다면, K-Fold Cross Validation
  * Data가 매우 적다면, 데이터 개수만큼 Cross Validation 수행(LOOCV)

#### 예제
GridSearchCV에서 cv 파라미터로 교차 검증을 사용할 수 있음

In [ ]:
'''
from sklearn.model_selection import GridSearchCV, KFold

grid_cv = GridSearchCV(Model_rf,
                       param_grid = params,
                       scoring = 'accuracy',
                       cv = KFold(n_splits = 5,
                                  random_state = 2045),
                       refit = True,
                       n_jobs = -1)'''

#### **랜덤 포레스트(Random Forest)**

### 앙상블(Ensemble)

여러가지 모델을 사용하여 Accuracy를 개선하는 방법

#### Ensemble의 종류


In [ ]:
filename = "Ensemble.png"
images_path = os.path.join(PROJECT_ROOT_DIR, "images")
show_img = mpimg.imread(os.path.join(images_path, filename))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.imshow(show_img)

#### **Random Forest(랜덤 포레스트)**

의사결정나무(Decision Tree)의 앙상블(Ensemble)

* 의사결정나무에서 나무를 여러개 만드는 것
* 지도학습의 일종

#### 특징
* 다수의 의사결정 나무들의 결과로부터 모델을 생성함
* 모델 생성 시 다양성(Diversity)과 임의성(Random) 부여
* 모델 Accuracy를 높이고 및 Overfitting 발생 가능성을 낮춤
* 일반적으로 의사결정나무 보다 성능이 좋음

### 1) 다양성(Diversity)과 임의성(Random)

In [ ]:
filename = "Diversity_random.png"
images_path = os.path.join(PROJECT_ROOT_DIR, "images")
show_img = mpimg.imread(os.path.join(images_path, filename))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.imshow(show_img)

#### 다양성(Diversity)

##### **Bagging(Bootstrap + Aggregating)**

나무를 다양하게 생성하고, 다양한 나무로부터 나온 결과들을 통합하는 것

##### **Bootstrap**

다양성을 위해 Bootstrap 방식으로 새로운 데이터를 생성함

* 원본 Data를 사용하여 여러 개의 서로 다른 Train Data를 생성함
  * 생성된 Train Data 마다 별도의 의사결정나무 모델을 생성함
  * 의사결정나무 모델 개수를 조정하기 위한 Hyperparameter : n_estimators
* Train Data는 Original Data에서 단순 복원 임의추출법으로 생성(중복이 있음)
  * 예를 들어 Original Data에 데이터포인트가 3개이고 nestimators가 3이라면,
    ($X_1, X_2, X_3$), ($X_1, X_1, X_3$), ($X_3, X_2, X_3$) 으로 Train data로부터 3개의 Bootstrap Data를 생성한다.

##### **Aggregating**

다양성을 위해 여러개 Bootstrap 모델의 결과를 통합함

* Hyperparameter인 모델의 개수(n_estimators)에 따라 통합 결과가 달라짐
* 모델별 통합 방법
  * 분류 모델 : 다수결 또는 가중치를 적용하여 통합
    * $\hat{y}=(1,0,0,0,1,1,1,1,1)=1$
      *홀수로 주지 않아도, 50대 50으로 나올 경우는 매우 드물기 때문에 짝수로 주어도 됨

  * 예측 모델 : 평균값 또는 가중평균값으로 통합
    * $\hat{y} = (77, 75, 76, 77, 76) = 76.2$


#### 임의성(Random)

##### **Random Subspace**

임의성을 위해 의사결정 나무 생성 시 Feature를 무작위로 선택하는 것

* 원본 Feature에서 무작위로 입력 Feature를 추출하여 적용함(하나의 입력에 Feature 중복이 없는 비복원 추출)
* Decision Tree 보다 다양한 Feature를 활용함
* 입력 변수 개수 조정을 위한 Hyperparameter : max_features
  * default : $\sqrt{Feature count}$

### 2) Hyperparameter

일반적으로 조정되는 파라미터는 상위 3개

##### **종류** 
1. n_estimators : 모델에 사용되는 의사결정나무 개수
2. max_features : 분할에 사용되는 Feature의 개수
3. max_depth : 트리 모델의 최대 깊이
4. max_leaf_nodes : Leaf 최대 개수
5. min_samples_split : 분할을 위한 최소한의 샘플 데이터 개수
6. min_samples_leaf : Leaf가 되기 위한 최소한의 샘플 데이터 개수

## **규제화(Regularization)**

#### **Overfitting**

모델이 Train 데이터에만 최적화된 상태

Loss가 Train Data에 대해서만 낮지만, Validation Data에 대해서는 높음 - 이걸 막아야 함

##### Overfitting 발생 원인
1. Data point 개수가 적을 때
2. Model Capacity가 높을 때
+ 파라미터($w_i$) 개수가 많은 경우

#### **Regulation(규제화)**

Model이 Train Data에 너무 학습되지 않도록 방해하는 것

Overfitting을 회피할 목적으로 파라미터 개수를 줄이는 방법

In [ ]:
filename = "regulation.png"
images_path = os.path.join(PROJECT_ROOT_DIR, "images")
show_img = mpimg.imread(os.path.join(images_path, filename))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.imshow(show_img)

* w의 차수가 크면 비용 함수가 구불구불해지면서 모든 데이터에 맞게 되는 Overfitting이 일어난다.
* 이러한 성질을 최소화 하기 위해 특정 값을 더하는데, 무엇을 더하는지에 따라 모델이 달라진다.
* 특정 값을 더하면 지속적으로 노이즈가 발생하여 Loss 증가하게 만든다.

#### **Ridge Regression(릿지 회귀)**

MSE 식에 $\alpha \sum\limits_{j=1}^m {w_j}^2$ 를 더해 w를 제어.

규제 방식 : L2

* $\alpha$ = 1(default)
* $\alpha$ 를 높이면 계수를 0에 더 가깝게 만들어 Train 모델의 성능은 나빠지지만 일반화에는 도움이 된다.
* $\alpha$ 를 낮추면 계수에 대한 제약이 풀리면서 과적합이 일어날 가능성이 증가한다.
* Test data에 대한 성능이 높아질 때까지 alpha 값을 줄이며 조절한다(Hyperparameter).
* 성능이 비슷하다면, 보통 Ridge와 Lasso 중 Ridge를 선호한다(어떤 계수도 0이 되지 않기 때문).

#### **Lasso Regression(라쏘 회귀)**

MSE 식에 $\alpha \sum\limits_{j=1}^m \left\vert {w_j} \right\vert$ 를 더해 w를 제약

규제 방식 : L1

* Ridge와 같이 계수를 0에 가깝게 만들려고 하는데, 방식이 조금 다르다.
* L1 규제 결과로 어떤 계수는 정말 0이 되는데, 모델에서 완전히 제외되는 Feature가 생긴다
* (Feature selection이 자동으로 이루어진다고 볼 수 있음).
* $\alpha$ = 1(default)
* $\alpha$ 를 높이면 계수를 0에 더 가깝게 만들어 Train 모델의 성능은 나빠지지만 일반화에는 도움이 된다.
* $\alpha$ 를 낮추면 계수에 대한 제약이 풀리면서 과적합이 일어날 가능성이 증가한다.
  * 예를 들어, 과소적합이 발생하여 $\alpha$ 를 줄일 때 max_iter(반복 실행하는 최대 횟수)의 기본값은 늘린다.
* Feature가 많고 그 일부만 중요하다면(0이 되는 경우 발생 고려) Lasso를 선호할 수 있다.

#### **ElasticNet(엘라스틱넷)**

Ridge와 Lasso를 결합한 회귀 $(l_1 \times \sum\limits_{j=1}^m \left\vert {w_j} \right\vert + \dfrac{1}{2} \times l_2 \times \sum\limits_{j=1}^m {w_j}^2) $

규제 방식 : L1 + L2

## **K-Nearest Neighbors(KNN)**

데이터 분류 시 이웃한 데이터 포인트의 분류를 바탕으로 하는 알고리즘

데이터의 Label을 정의할 때 주변 데이터들의 Label을 조사하여 다수결로 K개 이상이며, 가장 많은 것의 Label로 정의함

In [ ]:
filename = "knn.png"
images_path = os.path.join(PROJECT_ROOT_DIR, "images")
show_img = mpimg.imread(os.path.join(images_path, filename))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.imshow(show_img)

* K : 최근접 이웃의 개수, 하이퍼파라미터
* 최적의 K 값을 찾기 위해 파라미터를 튜닝(Cross Validation)
* 사기 탐지(Fraud Detection), 이상 감지(Anomaly Detection)에 적합
* 데이터 시각화를 통해 용이하게 분류 가능
  * 다차원 공간에서 계산량이 급격히 증가함
  * 예측자의 개수가 적고, 분류의 크기가 비슷할 때 사용 권장

## SMOTE(Over Sampling)

Target 데이터 수가 불균형할 때 상대적으로 부족한 데이터를 Over samplig(오버 샘플링) 하는 방법

* 이진 분류에서 Target 데이터 수가 불균형하여 SMOTE를 사용하면, 두 개의 Target 데이터 수는 똑같아 진다.

* Recall을 올리기 위해 사용했으나 Precision, F1-Score가 떨어질 수 있으므로 조심해서 사용해야 한다.

분류 실습 : 신용카드 사기 검출 (Credit Card Fraud Detection)

## **연관 규칙(Association Rules)**

데이터 사이의 연관된 규칙을 찾는 방법

특정 사건 발생 시 함께 자주 발생하는 다른 사건(조건부 확률)의 규칙(Rule)

* 비지도 학습의 일종
* 유사도를 측정하는 방법 중 하나
* 방향성이 존재함
  * Confidence, Lift : A $\rightarrow$ B (A를 산사람이 B도 구매)
* 예
  * 상품 A를 구매한 고객이 상품 B를 함께 구매할 확률(구매 패턴) 확인
  * 마케팅에서 고객별 장바구니 내 품목 간 관계 분석


### Support, Confidence, Lift

##### **Support**(지지도) - 전체 거래에 대한 A와 B가 동시에 발생할 확률

특정 품목 집합이 얼마나 자주 등장하는지 확인(예 : 빈번하게 판매되는 물품 확인)

* 특정 품목 집합을 포함하는 거래의 비율로 계산
* 방향성 없음(A $\rightarrow$ B == B $\rightarrow$ A)

$
support = \dfrac{\text{A와 B가 포함된 거래 수}}{\text{전체 거래 수}} $



##### **Confidence**(신뢰도)

상품 A가 존재할 때 상품 B가 나타나는 빈도

* 상품 A가 포함된 거래 중, 상품 B를 포함하는 거래 비율로 계산
* 방향성 있음(A(원인) $\rightarrow$ B(결과))

$
Confidence = \dfrac{\text{A(조건)와 B(결과)가 동시에 포함된 거래 수}}{\text{A(조건)를 포함한 거래 수}} $

* 감자칩을 사면 75% 확률로 맥주를 구매한다고 볼 수 있지만,
* 맥주의 판매 빈도를 고려하지 않고 감자칩의 판매 빈도만 고려했기 때문에 맥주 자체가 자주 거래되는 상품이라면 신뢰도를 부풀릴 수 있음
* Lift를 활용하여 두 물품 모두의 기반 빈도(Base Frequency)를 고려해야 함


##### Lift(향상도)

상품 A와 상품 B가 함께 팔리는 빈도

* 두 물품이 각각 얼마나 자주 거래되는지를 고려함
* 방향성 있음(A(원인) $\rightarrow$ B(결과))

* Lift 값
1 : 두 물품 사이에 연관성이 없음(독립 사건)
$< 1 $: 상품 A 거래 시 상품 B가 거래될 가능성 작음
$> 1 $: 상품 A 거래 시 상품 B가 거래될 가능성 있음

$
Lift = \frac{{Confidence(A \rightarrow B)}}{{Support(B)}}$
